In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation


%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [42]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [1000]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std

In [43]:
print(train_input.shape)
print(test_input.shape)
print(train_label.shape)
print(test_label.shape)

(7521, 4)
(7521, 4)
(7521, 2)
(89, 64)


In [44]:
nb_inputs  = 8
nb_hidden  = 96
nb_outputs = 2

time_step = 1e-3
nb_steps  = 100

batch_size = 128

import scipy.io

from dataset import Dataset, RegTorchSeasonalitySpikingDataset, RegSpikingDataset, RegTorchSpikingDataset
train_data = RegTorchSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
test_data = RegTorchSpikingDataset(test_input, train_label, nb_inputs, nb_steps)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
print(len(train_data))
print(train_data[0][0].shape)
print(train_data[0][1].shape)
print(train_data[0][2].shape)

7521
(32, 100)
(1,)
(2,)


In [5]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

       # neuron_params = {
       #         'threshold'     : 0.1,
       #         'current_decay' : 1,
       #         'voltage_decay' : 0.1,
       #         'requires_grad' : True,     
       #     }
        #neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        neuron_params = {
                'threshold'     : 1.00,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,     
            }
        
        
        self.blocks = torch.nn.ModuleList([
                #slayer.block.cuba.Input(neuron_params),
                slayer.block.cuba.Recurrent(neuron_params, input_size, 64, weight_norm=True, delay=True),
                #slayer.block.cuba.Dense(neuron_params, 32, 64, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 64, 128, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 128, output_size, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, input_size, 64, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 64, 128, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 128, output_size, weight_scale=2, weight_norm=True)
                #slayer.block.cuba.Recurrent(cuba_params, 100, 50),
                #slayer.block.cuba.KWTA(cuba_params, 50, 50, num_winners=5)
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16 + 1, 8)
        self.fc3 = nn.Linear(8, 2)
        #self.fc3 = nn.Linear(128, 128)
        #self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x, x1):
        x = x.flatten(start_dim=1)
        #x = torch.cat((x, x1), axis=1)
        x = self.fc1(x)
        x = self.act(x) 
        x = torch.cat((x, x1), axis=1)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        #x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        #x = self.act(x)
        #x = self.fc3(x)
        #x = self.act(x)
        #x = self.fc4(x)
        return x

In [6]:
net_snn = Network(nb_inputs * 4, 2).to(device)
net_dnn = DNNNetwork(2 * 100, 2).to(device)

for name, weight in net_snn.named_parameters():
    print(name, weight.shape)


blocks.0.neuron.current_decay torch.Size([1])
blocks.0.neuron.voltage_decay torch.Size([1])
blocks.0.delay.delay torch.Size([1])
blocks.0.input_synapse.weight_g torch.Size([64, 1, 1, 1, 1])
blocks.0.input_synapse.weight_v torch.Size([64, 32, 1, 1, 1])
blocks.0.recurrent_synapse.weight_g torch.Size([64, 1, 1, 1, 1])
blocks.0.recurrent_synapse.weight_v torch.Size([64, 64, 1, 1, 1])
blocks.1.neuron.current_decay torch.Size([1])
blocks.1.neuron.voltage_decay torch.Size([1])
blocks.1.delay.delay torch.Size([1])
blocks.1.synapse.weight_g torch.Size([128, 1, 1, 1, 1])
blocks.1.synapse.weight_v torch.Size([128, 64, 1, 1, 1])
blocks.2.neuron.current_decay torch.Size([1])
blocks.2.neuron.voltage_decay torch.Size([1])
blocks.2.synapse.weight_g torch.Size([2, 1, 1, 1, 1])
blocks.2.synapse.weight_v torch.Size([2, 128, 1, 1, 1])


In [7]:
def NewLoss(output, target):
    import torch
    loss = (output - target)**2 / (target + 1e-6)**2
    return torch.mean(loss)

def train(trainloader, testloader, model, DNN_model, lr=2e-3, nb_epochs=10):
    #params = [w1,w2]
    params = list(model.parameters()) + list(DNN_model.parameters())
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    loss_fn = torch.nn.MSELoss()
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    DNN_model.train()
    
    
    for e in range(nb_epochs):
        print(e)
        local_loss = []
        for x_local, x1_local, y_local in trainloader:
            x_local = x_local.float().to(device)
            x1_local = x1_local.float().to(device)
            y_local = y_local.float().to(device)
    
            #output = model(x_local)
            #output = output.flatten(start_dim=1)
            optimizer.zero_grad()
            output = model(x_local)
            output = DNN_model(output, x1_local)
            loss_val = 100 * loss_fn(output, y_local) 
            loss_val.backward()
            #print("AAAA: ", DNN_model.fc2.weight)
            #print("BBBB: ", DNN_model.fc1.weight.grad)
            optimizer.step()
            local_loss.append(loss_val.item())
            
        
        #if e % 1 == 30 and e != 0:
        #    print("Training accuracy: %.3f"%(compute_ber(trainloader, net, "train")))
        #    print("Test accuracy: %.3f"%(compute_ber(testloader, net, name)))
        scheduler.step()
        mean_loss = np.mean(local_loss)
        print(mean_loss)
        #print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        loss_hist.append(mean_loss)
        

In [8]:
train(train_loader, train_loader, net_snn, net_dnn, lr=1e-3, nb_epochs=100)

0
6.31429772457834
1
3.8061626927327303
2
3.2837029735920793
3
2.9969182499384477
4
2.683217173915798
5
1.9340117664660437
6
1.6032967072422222
7
1.4727632241734003
8
1.3543494454884932
9
1.2771226425292128
10
1.2182192034640555
11
1.136831973568868
12
1.0173073284706826
13
0.972050443039102
14
0.9325493159940688
15
0.8454138596179122
16
0.799137853969962
17
0.7888187202861754
18
0.829133358042119
19
0.7768537038463658
20
0.7699447961176856
21
0.716933203450704
22
0.6338880163128093
23
0.6714739248914233
24
0.6387739274966515
25
0.60731041431427
26
0.5700278362985385
27
0.5058621809644214
28
0.5340344327486167
29
0.5215734134791261
30
0.4888702307717275
31
0.4690013656676826
32
0.4747339666394864
33
0.4680277472835476
34
0.4434982659453053
35
0.41014546332723
36
0.4324578668101359
37
0.39919170813035154
38
0.4267957033747334
39
0.4036314505136619
40
0.38621606993473184
41
0.37252478730880606
42
0.3770100694086592
43
0.3923345172809342
44
0.37162195221852445
45
0.3688311551587056
46
0.4

In [28]:
net_snn.export_hdf5('./net_snn.net')
torch.save(net_dnn.state_dict(), './net_dnn.pth')

In [21]:
error = 0.0
inputs = []
labels = []
loss_fn = torch.nn.MSELoss()
all_loss =[]
test_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=True)
for input, input1, target in test_loader:
    inputs.append(input)
    labels.append(target)
    target = torch.tensor(target).float()
    input = torch.tensor(input).float()
    input1 = torch.tensor(input1).float()
    
    output = net_snn(input)
    output = net_dnn(output, input1)
    print(output.shape)
    #print(output, target)
    loss_val = loss_fn(output, target)
    all_loss.append(loss_val.item())
print(np.mean(all_loss))
inputs = np.array(inputs)
labels = np.array(labels)
#inputs = inputs.reshape((inputs.shape[0], -1))
print(inputs.shape)
rc.my_test()
np.save('input_snn.npy', inputs)
np.save('label_snn.npy', labels)

/tmp/ipykernel_77/3015334240.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_77/3015334240.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_77/3015334240.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()


torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])


KeyboardInterrupt: 

In [45]:
all_output = []
inputs = []
labels = []
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
for input, input1, target in test_loader:
    inputs.append(input)
    labels.append(target)
    target = torch.tensor(target).float()
    input = torch.tensor(input).float()
    input1 = torch.tensor(input1).float()
    
    output = net_snn(input)
    output = net_dnn(output, input1).cpu().detach().numpy()
    all_output.append(output)
    
all_output = np.concatenate(all_output, axis=0)
ber = rc.my_test(all_output)
print(ber)

/tmp/ipykernel_97/721791834.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_97/721791834.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_97/721791834.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()


0.20306647940074907


In [11]:
A1 = 8 * torch.ones((128, 8))
A2 = torch.ones((128, 1))
print(A1 + A2)

tensor([[9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        ...,
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.]])
